In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder  #for label encoding
from sklearn.metrics.cluster import homogeneity_score
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from numpy.linalg import norm


import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

from tensorflow.keras.layers import LSTM, MaxPool1D, Dropout, Dense, GlobalMaxPooling1D, Embedding, Activation
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

from sklearn.externals import joblib
from google.colab import files



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!




## OPENING TRAIN.CSV & VALID.CSV


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

def open_filter_data(filename):
    
    data = pd.read_csv("drive/MyDrive/SMAI_PROJECT_CLICKBAIT/"+filename+".csv", na_values='?', header=None,) 
    #data = pd.read_csv("clickbait-news-detection/"+filename+".csv", na_values='?', header=None,) 
    data.head()
    #print("data.shape before:",end=" ")
    #print(data.shape)
    
    df=data.dropna(axis = 0, how ='any')
    #print("data.shape after:",end=" ")
    #print(df.shape)
    df=df.reset_index(drop=True)
    return df
    

df_train=open_filter_data("train")
#df_train.describe

#print("******")

df_valid=open_filter_data("valid")
#df_valid.describe

df_test=open_filter_data("test")

Mounted at /content/drive


## PREPROCESSING OF TRAINING DATA & VALIDATION DATA

In [ ]:
def extract_data(df):
    a=df.iloc[1:,2]
    b=df.iloc[1:,1]
    df.iloc[:,3] = df.iloc[:,3].str.replace('news','1')
    df.iloc[:,3] = df.iloc[:,3].str.replace('clickbait','0')
    y_actual=df.iloc[1:,3]
    return a,b,y_actual

def extract_dd(df):
    a=df.iloc[1:,2]
    b=df.iloc[1:,1]
    return a,b
    

a_train,b_train,y_actual=extract_data(df_train)
a_valid,b_valid,y_valid=extract_data(df_valid)
a_test,b_test=extract_dd(df_test)

# print(a)
# print("**********")
# print(b)
# print("**********")
# print(y_actual)

# print("#################")

# print(a_valid)
# print("**********")
# print(b_valid)
# print("**********")
# print(y_valid)



1. Converting all sentences to lower case 
2. Writing Abbreviations in full form
3. Removing punctuations (Normalisation)
4. Removing Stop words (Normalisation)
5. Lemmatizing the data


In [ ]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
"'cause": "because", "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
"didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
"hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", "he'd": "he would",
"he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
"i'd": "I would", "i'd've": "I would have", "i'll": "I will", "i'll've": "I will have",
"i'm": "I am", "i've": "I have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is",
"let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have",
"mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock",
"oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not",
"shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not",
"shouldn't've": "should not have", "so've": "so have", "so's": "so is", "that'd": "that had",
"that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have",
"there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
"wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will",
"we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not",
"what'll": "what will", "what'll've": "what will have", "what're": "what are", "what's": "what is",
"what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
"where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why has", "why've": "why have",
"will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have",
"wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
"y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have", "you'd": "you would",
"you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"
}


def pre_process_data(a,b):
    final_lines=[]
    final_headings=[]


    for line in range(1,int(0.25*len(a))):
      #print(a[line])
      #print(type(a[line]))
      #print("*********")
      a[line]=(a[line]).lower()     #Converting all sentences to lower case 
      for word in a[line].split():
        if word in contractions:
            a[line]=a[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
      tokens = word_tokenize(a[line].lower()) 
      words = [word for word in tokens if word.isalpha()]    #Removing punctuations
      final_word = [w for w in words if not w in stop_words]     #Removing Stop words 
      final_words = [lemmatizer.lemmatize(w) for w in final_word]     #Lemmatizing words
      ans=""
      for x in final_words:
        ans= ans+ " "+x
      final_lines.append(ans.lstrip())
      b[line]=(b[line]).lower()     #Converting all sentences to lower case 
      for word in b[line].split():
        if word in contractions:
            b[line]=b[line].replace(word, contractions[word.lower()])  #Writing Abbreviations in full form
      tokens = word_tokenize(b[line].lower()) 
      words = [word for word in tokens if word.isalpha()]    #Removing punctuations
      final_head = [w for w in words if not w in stop_words]     #Removing Stop words 
      final_heads = [lemmatizer.lemmatize(w) for w in final_head]     #Lemmatizing words
      ans=""
      for x in final_heads:
        ans= ans+ " "+x
      final_headings.append(ans.lstrip())
    return final_headings,final_lines


def vectorisation(final_lines):    
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(final_lines)
    final_word_vector=[]
    i=0
    while (i<X.shape[0] and i+9000<X.shape[0]):
        final_word_vector.extend(X[i:i+9000].toarray())
        i+=9000

    final_word_vector.extend(X[i:].toarray())
    return final_word_vector


lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))



# pre-process all data

In [ ]:
final_headings,final_lines = pre_process_data(a_train,b_train)
final_headings_valid,final_body_valid = pre_process_data(a_valid,b_valid)
final_headings_test,final_body_test = pre_process_data(a_test,b_test)

In [ ]:
# print(len(final_headings),
#       len(final_lines),
#       len(final_headings_valid),
#       len(final_body_valid),
#       len(final_headings_test),
#       len(final_body_test)
# )

In [ ]:
final_headings.extend(final_lines)
final_headings.extend(final_headings_valid)
final_headings.extend(final_body_valid)
final_headings.extend(final_headings_test)
final_headings.extend(final_body_test)

In [ ]:
del stop_words
del lemmatizer
del final_lines



In [ ]:
print(len(final_headings))

In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(final_headings)]

In [ ]:
#temp=["hi I am Naman Juneja from India"]
#documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(temp)]

In [ ]:
#print(documents)

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
model = Doc2Vec(documents, vector_size=500, window=2, min_count=1, workers=4)

In [ ]:
print(model.docvecs[10])
x=[]
for i in range(len(model.docvecs)):
  x.append(model.docvecs[i])


[-6.59551763e-04  6.03501860e-04  3.52083036e-04 -7.86612072e-05
  4.49003157e-04 -5.99634543e-04 -3.77757446e-04 -8.22952832e-04
  1.64885045e-04  9.12347226e-04  5.63400856e-04  5.22610964e-04
  9.36096534e-04  7.52306951e-04 -2.37418339e-04  6.59546873e-04
 -9.74758936e-04 -6.41658553e-04 -1.17530355e-04  4.87689103e-04
  1.12697831e-04 -9.16111749e-04  7.73918058e-04 -4.86238539e-04
  4.40167903e-04  9.09582363e-04 -1.70210944e-04  1.91959276e-04
 -3.32270458e-04 -2.80366134e-04  4.92163934e-04 -1.83202574e-04
  9.01227235e-04 -6.26588706e-04 -5.48770244e-04  1.78484799e-04
  4.93132393e-04  9.42949962e-04 -9.25810367e-04  3.94707924e-04
 -3.54462449e-04  1.01759513e-04  4.17260802e-04 -8.16293003e-04
 -8.47954885e-04 -5.96403959e-04  3.13672528e-04 -7.72554951e-04
  4.19105083e-04  2.53702339e-04  8.35765502e-04 -7.68115511e-04
 -2.36163527e-04 -2.34607051e-05  7.82218936e-04  8.70700460e-04
  1.39000690e-07 -7.63974967e-04  2.99230829e-04 -2.33600396e-04
  6.40104045e-05 -4.07426

In [ ]:
x=np.array(x)
print(x.shape)

(14158, 500)


In [ ]:
print(model.docvecs[0])
print(x[0])

[-1.77043919e-02 -3.98352416e-03  3.32571822e-03  1.90945342e-02
  2.76227966e-02  1.08501280e-03 -1.76655408e-02  9.14620608e-03
 -3.39113995e-02 -3.89465922e-03  1.14468625e-02 -9.73985437e-03
 -1.85580514e-02  3.93938087e-02 -3.62894051e-02  1.55434916e-02
  1.45376977e-02 -1.15870815e-02  2.67695170e-02  3.50141786e-02
 -8.49125069e-03 -5.35152666e-03 -3.07664066e-03  2.27094386e-02
  1.37876682e-02  2.20297985e-02 -9.70258005e-03  1.80349629e-02
 -6.63314713e-03 -2.09383853e-02  3.88711691e-03 -1.17473565e-02
 -3.10280477e-03  1.37094250e-02  4.92659956e-03 -5.37831523e-03
  7.67016551e-03 -1.02280383e-03 -1.44093651e-02  2.04020832e-02
  3.52594703e-02  1.59889627e-02 -6.89812377e-03 -1.50766037e-02
 -5.13310428e-04 -2.23603174e-02  5.39254351e-03 -1.24264928e-02
 -1.54797481e-02 -2.81274654e-02 -1.20972609e-02  1.14014363e-02
  2.98199113e-02 -7.33224163e-03 -4.05118801e-03 -7.69557478e-03
  1.65393297e-02 -3.52382706e-03  1.35981254e-02  9.55768023e-03
  1.30503951e-02  1.63965

In [ ]:
#print(len(x))

In [ ]:
print(type(x))

<class 'numpy.ndarray'>


In [ ]:
final_heading_vector=np.array(x[0:4968])


In [ ]:
a=9936

final_body_vector=np.array(x[4968:a])
#  
# del final_body_vector


In [ ]:
print(final_body_vector.shape)

(4968, 500)


In [ ]:
validation_heading_vector=np.array(x[a:a+705])
#joblib.dump(validation_heading_vector,  'validation_heading_vector.pkl')
# del validation_heading_vector


In [ ]:
print(validation_heading_vector.shape)

(705, 500)


In [ ]:
validation_body_vector=np.array(x[a+705:a+1410])
#joblib.dump(validation_body_vector,  'validation_body_vector.pkl')
# del validation_body_vector


In [ ]:
test_heading_vector=np.array(x[a+1410:a+2816])
#joblib.dump(test_heading_vector,  'test_heading_vector.pkl')
# del test_heading_vector


In [ ]:
print(test_heading_vector.shape)

(1406, 500)


In [ ]:
test_body_vector=np.array(x[a+2816:])
#joblib.dump(test_body_vector, 'test_body_vector.pkl')
# del test_body_vector

In [ ]:
print(test_body_vector.shape)

(1406, 500)


In [ ]:
print(validation_heading_vector.shape)
print(validation_body_vector.shape)
print(y_valid.shape)


(705, 500)
(705, 500)
(2824,)


# for testing data

In [ ]:
# print(len(final_heading_vector))
# print(len(final_body_vector))
# print(len(validation_heading_vector))
# print(len(validation_body_vector))
# print(len(test_heading_vector))
# print(len(test_body_vector))



In [ ]:
y_actual=y_actual[:4968].astype(float)
y_valid=y_valid[:705].astype(float)


In [ ]:
y_actual=np.array(y_actual)
y_valid=np.array(y_valid)

In [ ]:
# joblib.dump(y_actual,  'y_actual.pkl')
# #del y_actual

# joblib.dump(y_valid,  'y_valid.pkl')
# #del y_valid

## MODEL

In [ ]:
print(final_heading_vector.shape)
print(final_body_vector.shape)
print(y_actual.shape)

print("----------------------")

print(validation_heading_vector.shape)
print(validation_body_vector.shape)
print(y_valid.shape)

print("----------------------")

print(test_heading_vector.shape)
print(test_body_vector.shape)
print("----------------------")



(4968, 500)
(4968, 500)
(4968,)
----------------------
(705, 500)
(705, 500)
(705,)
----------------------
(1406, 500)
(1406, 500)
----------------------


In [ ]:
from keras.layers import LSTM
def create_base_network(input_shape):
    input = Input(shape=input_shape)
    """
    x = LSTM(64, activation='softmax')(input)
    x = Dropout(0.1)(x)
    """
    x = Dense(32, activation='relu')(input)
    x = Dropout(0.1)(x)
    x = Dense(32, activation='relu')(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    return Model(input, x)


def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    
    margin = 1.0
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return float(K.mean(float(y_true) * float(sqaure_pred) + (1.0 - float(y_true)) * float(margin_square)))



In [ ]:
from numpy import dot
from numpy.linalg import norm
from keras.backend import int_shape
def cosine_sim(vects):
  a,b=vects
  #c=tf.constant(a)
  ans=[]
  s=int_shape(a)[1]
  for i in range(s):
    temp1=a[i]
    temp2=b[i]
    normalize_a = tf.nn.l2_normalize(temp1,0)        
    normalize_b = tf.nn.l2_normalize(temp2,0)
    cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
    ans.append(cos_similarity)
  return tf.convert_to_tensor(ans)
  

In [ ]:
from sklearn.preprocessing import normalize
def func(vects):
  a,b=vects
  t1=tf.shape(a)
  s1=t1[0]
  s2=t1[1]
  a=tf.make_ndarray(a)
  b=tf.make_ndarray(b)
  s1=a.shape[0]
  s2=a.shape[1]
  ans=[]
  print(a.shape[0],a.shape[1])
  for i in range(s1):
    summ=0
    a[i]=tf.make_ndarray(a[i])
    b[i]=tf.make_ndarray(b[i])
    temp1=a[i]/np.linalg.norm(a[i])
    temp2=b[i]/np.linalg.norm(b[i])
    #print(temp1,temp2)
    for j in range(s2):
      #print(a[i][j],b[i][j])
      summ+=temp1[j]*temp2[j]
    ans.append(summ)
  return tf.convert_to_tensor(ans)


In [ ]:
def accuracy(y_true, y_pred):
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:
def compute_accuracy(y_true, y_pred):
  pred = y_pred.ravel() < 0.0
  return np.mean(pred == y_true)


In [ ]:
import keras
# network definition
input_shape = final_heading_vector.shape[1]
ipt_shape=(final_heading_vector.shape[1],1)
base_network = create_base_network(input_shape)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches

processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(cosine_sim,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model([input_a, input_b], distance)

# train
#rms = RMSprop()
rms = keras.optimizers.Adam()
#rms = SGD()

model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])
# train
fh=final_heading_vector.reshape((final_heading_vector.shape[0],final_heading_vector.shape[1],1))
fb=final_body_vector.reshape((final_body_vector.shape[0],final_body_vector.shape[1],1))
f1=validation_heading_vector.reshape((validation_heading_vector.shape[0],validation_heading_vector.shape[1],1))
f2=validation_body_vector.reshape((validation_body_vector.shape[0],validation_body_vector.shape[1],1))
y_actual=y_actual.reshape((y_actual.shape[0],1))
y_valid=y_valid.reshape((y_valid.shape[0],1))
history = model.fit([fh[:4928], fb[:4928]], y_actual[:4928],
          batch_size=32,
          epochs=5,
          validation_data=([f1[:704],f2[:704]], y_valid[:704]))

#te_acc = compute_accuracy(y_test, y_pred_tr)

#print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
#print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))


Epoch 1/5
154/154 [==============================] - 7s 10ms/step - loss: 0.2330 - accuracy: 0.6559 - val_loss: 0.2020 - val_accuracy: 0.7344
Epoch 2/5
154/154 [==============================] - 1s 4ms/step - loss: 0.1999 - accuracy: 0.7151 - val_loss: 0.2065 - val_accuracy: 0.7017
Epoch 3/5
154/154 [==============================] - 1s 4ms/step - loss: 0.1959 - accuracy: 0.7195 - val_loss: 0.2027 - val_accuracy: 0.7230
Epoch 4/5
154/154 [==============================] - 1s 4ms/step - loss: 0.1844 - accuracy: 0.7397 - val_loss: 0.2037 - val_accuracy: 0.7202
Epoch 5/5
154/154 [==============================] - 1s 4ms/step - loss: 0.1874 - accuracy: 0.7322 - val_loss: 0.2073 - val_accuracy: 0.7102


In [ ]:
fh.shape

(4968, 500, 1)

In [ ]:
print(input_shape)

(500,)


In [ ]:
y_valid.shape

(705, 1)

In [ ]:
np.count_nonzero(y_valid)

524

In [ ]:
print(validation_body_vector.shape)
print(y_valid.shape)

(705, 500)
(705, 1)


In [ ]:
y_pred = model.predict([f1[:704],f2[:704]])
#tr_acc = compute_accuracy(y_valid, y_pred)
print(y_pred.shape)
print(y_pred)


(704,)
[0.7524947  0.5558719  0.67856824 0.4076063  0.78091216 0.33773738
 0.4844525  0.15765953 0.5061254  0.37648028 0.3519172  0.379403
 0.16965643 0.36533165 0.43413118 0.3608508  0.36660305 0.35220647
 0.15400147 0.12205665 0.54012084 0.3315304  0.08934474 0.21897951
 0.3750313  0.3098152  0.24186905 0.1420366  0.07894354 0.38685143
 0.28574497 0.13824776 0.20884013 0.1494909  0.03625047 0.26822934
 0.1462081  0.16237547 0.08649704 0.12529081 0.10070957 0.3920771
 0.21812531 0.09450607 0.09782926 0.03309109 0.56577736 0.11187753
 0.12710816 0.29319692 0.10574586 0.08539338 0.2818669  0.2715388
 0.2820727  0.28401285 0.07617581 0.17015556 0.35524112 0.4709994
 0.22394122 0.19622728 0.28873724 0.15455784 0.11667557 0.03495875
 0.18788925 0.11077882 0.45700273 0.11711904 0.09061936 0.20596468
 0.2280556  0.1734901  0.13509154 0.05225857 0.2512681  0.05682894
 0.5171086  0.09821615 0.18785702 0.22058539 0.13557781 0.14169003
 0.15682258 0.15376037 0.09224221 0.19948968 0.14483671 0.05

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 500)]        0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 64)           36224       input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (64,)                0           model[0][0]                

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
b = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_b")
normalize_a = tf.nn.l2_normalize(a,0)        
normalize_b = tf.nn.l2_normalize(b,0)
cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
sess=tf.Session()
cos_sim=sess.run(cos_similarity,feed_dict={a:[[1],[1]],b:})
print(cos_sim)


In [ ]:
ans1=func(validation_body_vector,validation_heading_vector)
ans1=np.array(ans1)
print(ans1)
print(len(ans1))

In [ ]:
ans1=func(np.array([[1,0]]),np.array([[1,1]]))
ans1=np.array(ans1)
print(ans1)
print(len(ans1))